# Module 5.2 - Predictive Maintenance Analysis for Semiconductor Equipment

This notebook demonstrates predictive maintenance techniques for semiconductor manufacturing equipment using time-series sensor data and engineered features.

## Learning Objectives
- Generate synthetic semiconductor equipment sensor data with degradation patterns
- Engineer time-window features for predictive maintenance
- Implement classification and regression models for maintenance prediction
- Evaluate models using manufacturing-specific metrics
- Optimize thresholds for early warning systems
- Understand temporal validation and data leakage prevention

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Environment setup complete!")

## 1. Data Generation and Understanding

We'll use the pipeline script to generate synthetic semiconductor equipment data that simulates:
- Multiple tools with different sensor configurations
- Gradual degradation patterns leading to maintenance events
- Realistic sensor noise and drift
- Time-based structure for proper temporal analysis

In [ ]:
# Use the pipeline script to generate and analyze data
import subprocess
import json

# Train a model to see the data characteristics
result = subprocess.run([
    'python', '5.2-predictive-maintenance-pipeline.py', 'train', 
    '--model', 'logistic', '--n-estimators', '10'  # Quick training for demo
], capture_output=True, text=True)

if result.returncode == 0:
    output = json.loads(result.stdout)
    print(f"Dataset characteristics:")
    print(f"  Samples: {output['n_samples']:,}")
    print(f"  Features: {output['n_features']}")
    print(f"  Model: {output['model']}")
    print(f"  Task: {output['task']}")
    print(f"  ROC AUC: {output['metrics']['roc_auc']:.3f}")
    print(f"  PR AUC: {output['metrics']['pr_auc']:.3f}")
    print(f"  PWS: {output['metrics']['pws']:.3f}")
else:
    print(f"Error: {result.stderr}")

## 2. Model Comparison

Let's compare different models for predictive maintenance.

In [ ]:
# Compare different models
models = ['logistic', 'rf', 'xgboost']
results = {}

for model in models:
    try:
        result = subprocess.run([
            'python', '5.2-predictive-maintenance-pipeline.py', 'train',
            '--model', model, '--n-estimators', '50'
        ], capture_output=True, text=True, timeout=120)
        
        if result.returncode == 0:
            output = json.loads(result.stdout)
            results[model] = output['metrics']
            print(f"{model.upper():>10}: ROC AUC={output['metrics']['roc_auc']:.3f}, PWS={output['metrics']['pws']:.3f}")
        else:
            print(f"{model.upper():>10}: Error - {result.stderr[:100]}")
    except subprocess.TimeoutExpired:
        print(f"{model.upper():>10}: Timeout")
    except Exception as e:
        print(f"{model.upper():>10}: Exception - {str(e)}")

## 3. Regression Analysis (Time-to-Event)

Now let's look at regression models for predicting time until maintenance.

In [ ]:
# Train regression model
result = subprocess.run([
    'python', '5.2-predictive-maintenance-pipeline.py', 'train',
    '--task', 'regression', '--target', 'time_to_event',
    '--model', 'rf', '--n-estimators', '50'
], capture_output=True, text=True)

if result.returncode == 0:
    output = json.loads(result.stdout)
    print(f"Regression Model Performance:")
    print(f"  MAE: {output['metrics']['mae']:.2f} hours")
    print(f"  RMSE: {output['metrics']['rmse']:.2f} hours")
    print(f"  R²: {output['metrics']['r2']:.3f}")
    print(f"  PWS (±24h): {output['metrics']['pws']:.3f}")
    print(f"  Estimated Loss: {output['metrics']['estimated_loss']:.2f}")
else:
    print(f"Error: {result.stderr}")

## 4. Model Persistence and Evaluation

Let's train a model, save it, and then evaluate it to demonstrate the full pipeline.

In [ ]:
import tempfile
import os

# Create temporary file for model
with tempfile.NamedTemporaryFile(suffix='.joblib', delete=False) as tmp_file:
    model_path = tmp_file.name

try:
    # Train and save model
    result = subprocess.run([
        'python', '5.2-predictive-maintenance-pipeline.py', 'train',
        '--model', 'rf', '--n-estimators', '50',
        '--save', model_path
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        print("Model trained and saved successfully!")
        
        # Evaluate the saved model
        eval_result = subprocess.run([
            'python', '5.2-predictive-maintenance-pipeline.py', 'evaluate',
            '--model-path', model_path
        ], capture_output=True, text=True)
        
        if eval_result.returncode == 0:
            eval_output = json.loads(eval_result.stdout)
            print("\nModel evaluation:")
            print(f"  ROC AUC: {eval_output['metrics']['roc_auc']:.3f}")
            print(f"  PR AUC: {eval_output['metrics']['pr_auc']:.3f}")
            print(f"  PWS: {eval_output['metrics']['pws']:.3f}")
        else:
            print(f"Evaluation error: {eval_result.stderr}")
    else:
        print(f"Training error: {result.stderr}")
        
finally:
    # Clean up
    if os.path.exists(model_path):
        os.unlink(model_path)

## 5. Key Insights and Recommendations

Based on the analysis above, here are the key insights:

1. **Feature Engineering**: Time-window features (rolling statistics, EWMA, lag features) are crucial for capturing degradation patterns

2. **Model Performance**: Tree-based models (Random Forest, XGBoost) generally perform better than linear models for this type of time series prediction

3. **Manufacturing Metrics**: PWS (Prediction Within Spec) and Estimated Loss provide business-relevant performance measures beyond standard ML metrics

4. **Temporal Validation**: Proper time-based splitting with embargo periods is essential to prevent data leakage

5. **Class Imbalance**: Maintenance events are rare, requiring careful handling through class weights or resampling

## Next Steps

For production deployment:
- Implement real-time feature computation
- Set up model monitoring and retraining pipelines
- Integrate with manufacturing execution systems (MES)
- Establish alert thresholds based on business costs
- Validate on real equipment data